In [1]:
!pip -V

pip 23.3.1 from /root/feature-map-experiments/.cenv/lib/python3.10/site-packages/pip (python 3.10)


# .init

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
import os
import sys
from pathlib import Path
workspace_dir = Path.cwd()
if not (workspace_dir / '.git').exists():
    workspace_dir = workspace_dir.parent


module_path = str(workspace_dir.resolve()) # os.path.abspath(os.path.join('..'))
print(f'workspace dir: {module_path}')

# Fixes a periodic issue with locating local packages.
if module_path not in sys.path:
    sys.path.append(module_path)

workspace dir: /root/feature-map-experiments


In [4]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

In [5]:
# import openslide
slides_dir = workspace_dir / 'WSI'
# slide_path = slides_dir / 'C-16-098.svs'

wsi_file_name = 'C-17-036-2021-07-15_215134.ndpi'
slide_path = slides_dir / wsi_file_name

print(f'{slide_path}')


/root/feature-map-experiments/WSI/C-17-036-2021-07-15_215134.ndpi


In [6]:
slide_path.suffix[1:]

'ndpi'

In [7]:
from fmc.file_cache import CachcedFileIndex, OpenSlideWSIDriver

index = CachcedFileIndex(index_root=workspace_dir/'.indexes')

wsi_driver = OpenSlideWSIDriver(file_index=index, original_files_path=slides_dir)
wsi_driver.save_metadata(wsi_file_name)

# Extract patches

In [8]:
wsi_driver.generate_patches(wsi_file_name)

width=134400, height=64512, cols: 150 rows: 72
Total: 1000, Saved: 105, blank: 895
Total: 1243, Saved: 229, blank: 1014
